## AI-CA1-Search

**Name: Sina Tabassi**

**ID: 810199554**

## Modeling Explanation
For modeling this problem, we used state as our main search target that has a lot of features. Its arguments are pos,spent_time,waiting_time,path,imapassble_nodes,found_recipes,satisfied,disciples.<br>
our search will be on states, not the nodes. we use nodes as our path to visit and explore the states and this method allows to pass same node if our state changes.


## Modeling The Problem
- **Initial state**: we use state and pass its arguments as our first step of algorthim: 
  state(pos,spent_time,waiting_time,path,impassable_nodes,found_recipes,satisfied_disciples)
  
- **Actions**: actions of BFS and IDS algorithms are similar, but the actions in A* and weighted A* algorthims are a little different.

    - **BFS & IDS Actions**: we have a queue of frontier states and in every step we add the neighbors of the last node to first of our queue.
    
    - **A* & weighted_A* Actions**: we have a min-heap of frontier states that is ordered first by their priority and second by the order of adding them in to our min-heap. In every step we explore the first node poped from the min-heap.
    
- **Goal state**: The goal is to satisfy all disciples and the waiting_time is 0.

- **Path cost** : our spent_time is our path cost.

## Improts

In [5]:
from collections import deque
from dataclasses import dataclass
import time
import heapq

## State Class

In [6]:
@dataclass
class State:
    def __init__(self, pos, spent_time, waiting_time, path, impassable_nodes, found_recipes, satisfied_disciple):
        self.pos = pos
        self.spent_time = spent_time
        self.waiting_time = waiting_time
        self.satisfied_disciple = satisfied_disciple
        self.path = path
        self.found_recipes = found_recipes
        self.impassable_nodes = impassable_nodes


    def __eq__(self, other):
        if isinstance(other, State) == True:
            if other.pos == self.pos and other.found_recipes == self.found_recipes and other.satisfied_disciple == self.satisfied_disciple:
                return True
        return False

## Options Class

In [7]:
class Options():
    def execution_time_calculater(self, func):
        def wrap_func(*args,**kwargs):
            begin = time.time()
            result = func(*args,**kwargs)
            end = time.time()
            total_time =end - begin
            print(f'execution time: {total_time}s\n')
            return result
        return wrap_func
    
    def print_result(self, state, state_counter):
        print("Path :", ' -> '.join(map(str, state.path)))
        print("Total Number of States :", state_counter)
        print("Total Time :", state.spent_time)




options = Options()

## Read File

In [8]:
def read_file(file_name):
    global neighbors,  disciple_nodes_list, seyed_pos, impassable_nodes_list, recipe_nodes_list, disciple_nodes_num, recipe_nodes_num,\
    disciple_recipes, impassable_nodes
    neighbors =[]
    impassable_nodes_list = []
    recipe_nodes_list = []
    disciple_nodes_list = []
    seyed_pos = 0
    disciple_nodes_num = 0
    recipe_nodes_num = 0
    disciple_recipes = dict()
    impassable_nodes = dict()

    with open(file_name, 'r') as test_file:
        line1 = test_file.readline().split()
        vertices = int(line1[0])
        edges = int(line1[1])

        neighbors = [[] for i in range(vertices)]
        for i in range(edges):
            line2 = test_file.readline().split()
            pos1 = int(line2[0])
            pos2 = int(line2[1])
            neighbors[pos1 - 1].append(pos2)
            neighbors[pos2 - 1].append(pos1)

        for i in range(vertices):
            impassable_nodes_list.append(0)
            recipe_nodes_list.append(0)
            disciple_nodes_list.append(0)

        hard_nodes_num = int(test_file.readline())
        line3 = test_file.readline().split()
        for i in range(hard_nodes_num):
            hard_node_position = int(line3[i])
            impassable_nodes_list[hard_node_position - 1] = 1
            impassable_nodes[hard_node_position] = 0

        disciple_nodes_num = int(test_file.readline())
        for i in range(disciple_nodes_num):
            line4 = test_file.readline().split()
            disciple_pos = int(line4[0])
            disciple_recipes[disciple_pos] = set()
            disciple_nodes_list[disciple_pos - 1] = 3

            recipes_num = int(line4[1])
            for j in range(2, recipes_num+2):
                recipe_position = int(line4[j])
                recipe_nodes_list[recipe_position - 1] = 2
                disciple_recipes[disciple_pos].add(recipe_position)

        seyed_pos = int(test_file.readline())

    for node in recipe_nodes_list:
        if node == 2:
            recipe_nodes_num += 1

## Goal Check

In [9]:
def reaching_goal_check(state):
    if len(state.satisfied_disciple) == disciple_nodes_num and state.waiting_time == 0:
        return True
    else:
        False

## BFS Algorithm

In [14]:
@options.execution_time_calculater
def bfs(file_name):
    explored_states = []
    state_counter = 1

    my_file = file_name.replace('.txt', '')
    read_file(file_name)
    print(f"testing bfs algorithm for testcase({my_file})")

    frontier_states = deque()
    first_state = State(seyed_pos, 0, 0, [seyed_pos], impassable_nodes, [], [])
    frontier_states.appendleft(first_state)

    while (True):
        state = frontier_states.pop()

        if reaching_goal_check(state) == True:
            options.print_result(state, state_counter)
            return

        if state.waiting_time != 0:
            state.spent_time += 1
            state.waiting_time -= 1
            frontier_states.appendleft(state)
            continue

        explored_states.append(state)
        neighbors_pos = neighbors[state.pos - 1]

        for each_neighbor_pos in neighbors_pos:
            waiting_time = 0
            another_disciple_list = state.satisfied_disciple
            another_recipes_list = state.found_recipes
            another_impassable_nodes_list = state.impassable_nodes.copy()
            new_path = state.path.copy()
            new_path.append(each_neighbor_pos)


            if recipe_nodes_list[each_neighbor_pos - 1] == 2:
                another_recipes_list = set(another_recipes_list)
                another_recipes_list.add(each_neighbor_pos)

            if disciple_nodes_list[each_neighbor_pos - 1] == 3:
                if disciple_recipes[each_neighbor_pos].issubset(another_recipes_list):
                    another_disciple_list = set(another_disciple_list)
                    another_disciple_list.add(each_neighbor_pos)

            if impassable_nodes_list[each_neighbor_pos - 1] == 1:
                waiting_time = another_impassable_nodes_list[each_neighbor_pos]
                another_impassable_nodes_list[each_neighbor_pos] += 1

            new_state = State(each_neighbor_pos, state.spent_time + 1, waiting_time, new_path, another_impassable_nodes_list,
                              another_recipes_list, another_disciple_list)

            if reaching_goal_check(new_state) == True:
                options.print_result(new_state, state_counter)
                return

            if new_state not in explored_states:
                if new_state not in frontier_states:
                    state_counter += 1
                    frontier_states.appendleft(new_state)

## Running BFS Algorithm on Testcases

**1st Testcase:**

In [97]:
bfs("input1.txt")

testing bfs algorithm for testcase(input1)
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Total Number of States : 33
Total Time : 8
execution time: 0.0014805793762207031s



**2nd Testcase:**

In [98]:
bfs("input2.txt")

testing bfs algorithm for testcase(input2)
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Total Number of States : 88
Total Time : 8
execution time: 0.003000497817993164s



**3rd Testcase:**

In [99]:
bfs("input3.txt")

testing bfs algorithm for testcase(input3)
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Total Number of States : 844
Total Time : 13
execution time: 0.14405608177185059s



## IDS Algorithm

In [100]:
@options.execution_time_calculater
def ids(file_name):
    frontier_states = []
    state_counter = 1
    depth = 1

    my_file = file_name.replace('.txt', '')
    read_file(file_name)
    print(f"testing ids algorithm for testcase({my_file})")

    while (True):
        if len(frontier_states) == 0:
            first_state = State(seyed_pos, 0, 0, [seyed_pos], impassable_nodes, [], [])
            frontier_states = []
            frontier_states.append(first_state)
            depth += 1

        state = frontier_states.pop()

        if state.spent_time > depth:
            continue

        if state.waiting_time != 0:
            state.spent_time += 1
            state.waiting_time -= 1
            frontier_states.append(state)
            continue

        neighbors_pos = neighbors[state.pos - 1]
        neighbors_pos.reverse()

        for each_neighbor_pos in neighbors_pos:
            waiting_time = 0
            another_impassable_nodes_list = state.impassable_nodes.copy()
            another_disciple_list = state.satisfied_disciple
            another_recipes_list = state.found_recipes
            new_path = state.path.copy()
            new_path.append(each_neighbor_pos)

            if recipe_nodes_list[each_neighbor_pos - 1] == 2:
                another_recipes_list = set(another_recipes_list)
                another_recipes_list.add(each_neighbor_pos)

            if disciple_nodes_list[each_neighbor_pos - 1] == 3:
                if disciple_recipes[each_neighbor_pos].issubset(another_recipes_list):
                    another_disciple_list = set(another_disciple_list)
                    another_disciple_list.add(each_neighbor_pos)

            if impassable_nodes_list[each_neighbor_pos - 1] == 1:
                waiting_time = another_impassable_nodes_list[each_neighbor_pos]
                another_impassable_nodes_list[each_neighbor_pos] += 1

            new_state = State(each_neighbor_pos, state.spent_time + 1, waiting_time, new_path, another_impassable_nodes_list,
                              another_recipes_list, another_disciple_list)

            if reaching_goal_check(new_state) == True:
                options.print_result(new_state, state_counter)
                return

            if new_state not in frontier_states:
                state_counter += 1
                frontier_states.append(new_state)


## Running IDS Algorithm on Testcases

**1st Testcase:**

In [101]:
ids("input1.txt")

testing ids algorithm for testcase(input1)
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Total Number of States : 914
Total Time : 8
execution time: 0.00403904914855957s



**2nd Testcase:**

In [102]:
ids("input2.txt")

testing ids algorithm for testcase(input2)
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Total Number of States : 3793
Total Time : 8
execution time: 0.01401209831237793s



**3rd Testcase:**

In [103]:
ids("input3.txt")

testing ids algorithm for testcase(input3)
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Total Number of States : 68476
Total Time : 13
execution time: 0.21404528617858887s



## Heuristic function
heuristic function : number of disciple and recipe nodes that are not statisfied

## Consistency of Heuristic function
and for this heruristic to be consistant , we use spent_time and heuristic function, that means we use f(n) = g(n) + h(n) for pushing in min-heap and use A* algorithm. it is consistant because for all Ai that are parent of node Aj (both are in path like path = [A1 , A2 , ... , Ai , ... , Aj , ... , An]) we have :

h(A{i} - A{j}) <=  cost(A{i} to A{j})

that cause of we use this heuristic function: f(n) = g(n) + h(n).

## Heuristic

In [104]:
def heuristic(state):
    return recipe_nodes_num - len(state.found_recipes) + disciple_nodes_num - len(state.satisfied_disciple) + state.waiting_time

## A* Algorithm

In [105]:
@options.execution_time_calculater
def a_star(file_name):
    frontier_states = []
    explored_states = []
    state_counter = 1
    heapq.heapify(frontier_states)

    read_file(file_name)
    my_file = file_name.replace('.txt', '')
    print(f"testing A* algorithm for testcase({my_file})")


    first_state = State(seyed_pos, 0, 0, [seyed_pos], impassable_nodes, [], [])
    heapq.heappush(frontier_states, (heuristic(first_state) + 0, 0, first_state))

    while (True):
        state = heapq.heappop(frontier_states)[2]

        if reaching_goal_check(state) == True:
            options.print_result(state, state_counter)
            return

        if state.waiting_time != 0:
            state.spent_time += 1
            state.waiting_time -= 1
            heapq.heappush(frontier_states, (heuristic(state) + state.spent_time, state_counter, state))
            state_counter += 1

        neighbors_pos = neighbors[state.pos - 1]
        explored_states.append(state)

        for each_neighbor_pos in neighbors_pos:
            waiting_time = 0
            another_impassable_nodes_list = state.impassable_nodes.copy()
            another_disciple_list = state.satisfied_disciple
            another_recipes_list = state.found_recipes
            new_path = state.path.copy()
            new_path.append(each_neighbor_pos)

            if recipe_nodes_list[each_neighbor_pos - 1] == 2:
                another_recipes_list = set(another_recipes_list)
                another_recipes_list.add(each_neighbor_pos)

            if disciple_nodes_list[each_neighbor_pos - 1] == 3:
                if disciple_recipes[each_neighbor_pos].issubset(another_recipes_list):
                    another_disciple_list = set(another_disciple_list)
                    another_disciple_list.add(each_neighbor_pos)

            if impassable_nodes_list[each_neighbor_pos - 1] == 1:
                waiting_time = another_impassable_nodes_list[each_neighbor_pos]
                another_impassable_nodes_list[each_neighbor_pos] += 1

            new_state = State(each_neighbor_pos, state.spent_time + 1, waiting_time, new_path, another_impassable_nodes_list,
                              another_recipes_list, another_disciple_list)

            if reaching_goal_check(new_state) == True:
                options.print_result(new_state, state_counter)
                return

            if new_state not in explored_states:
                frontier_check = False
                for frontier_state in frontier_states:
                    if new_state == frontier_state[2]:
                        frontier_check = True
                        break

                if frontier_check == False:
                    heapq.heappush(frontier_states,(heuristic(new_state) + new_state.spent_time, state_counter, new_state))
                    state_counter += 1

## Running A* Algorithm on Testcases

**1st Testcase:**

In [106]:
a_star("input1.txt")

testing A* algorithm for testcase(input1)
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Total Number of States : 34
Total Time : 8
execution time: 0.002003192901611328s



**2nd Testcase:**

In [107]:
a_star("input2.txt")

testing A* algorithm for testcase(input2)
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Total Number of States : 53
Total Time : 8
execution time: 0.0009860992431640625s



**3rd Testcase:**

In [108]:
a_star("input3.txt")

testing A* algorithm for testcase(input3)
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Total Number of States : 459
Total Time : 13
execution time: 0.026429176330566406s



## Weighted A* Algorithm

In [109]:
@options.execution_time_calculater
def weighted_a_star(alpha, file_name):

    frontier_states = []
    explored_states = []
    state_counter = 1
    heapq.heapify(frontier_states)

    my_file = file_name.replace('.txt', '')
    read_file(file_name)
    print(f"testing weighted A* algorithm for testcase({my_file})")

    first_state = State(seyed_pos, 0, 0, [seyed_pos], impassable_nodes, [], [])
    heapq.heappush(frontier_states, (heuristic(first_state) * alpha + 0, 0, first_state))

    while (True):

        state = heapq.heappop(frontier_states)[2]
        if reaching_goal_check(state) == True:
            options.print_result(state, state_counter)
            return

        if state.waiting_time != 0:
            state.spent_time += 1
            state.waiting_time -= 1
            heapq.heappush(frontier_states,(heuristic(state) * alpha + state.spent_time, state_counter, state))
            state_counter += 1


        neighbors_pos = neighbors[state.pos - 1]
        explored_states.append(state)

        for each_neighbor_pos in neighbors_pos:
            waiting_time = 0
            another_impassable_nodes_list = state.impassable_nodes.copy()
            another_disciple_list = state.satisfied_disciple
            another_recipes_list = state.found_recipes
            new_path = state.path.copy()
            new_path.append(each_neighbor_pos)

            if recipe_nodes_list[each_neighbor_pos - 1] == 2:
                another_recipes_list = set(another_recipes_list)
                another_recipes_list.add(each_neighbor_pos)

            if disciple_nodes_list[each_neighbor_pos - 1] == 3:
                if disciple_recipes[each_neighbor_pos].issubset(another_recipes_list):
                    another_disciple_list = set(another_disciple_list)
                    another_disciple_list.add(each_neighbor_pos)

            if impassable_nodes_list[each_neighbor_pos - 1] == 1:
                waiting_time = another_impassable_nodes_list[each_neighbor_pos]
                another_impassable_nodes_list[each_neighbor_pos] += 1

            new_state = State(each_neighbor_pos, state.spent_time + 1, waiting_time, new_path, another_impassable_nodes_list,
                              another_recipes_list, another_disciple_list)

            if reaching_goal_check(new_state) == True:
                options.print_result(new_state, state_counter)
                return

            if new_state not in explored_states:
                frontier_check = False
                for frontier_state in frontier_states:
                    if new_state == frontier_state[2]:
                        frontier_check = True
                        break

                if  frontier_check == False:
                    heapq.heappush(frontier_states,(heuristic(new_state) * alpha + new_state.spent_time, state_counter, new_state))
                    state_counter += 1

## Running Weighted A* Algorithm on Testcases

### Alpha = 2

**1st Testcase:**

In [110]:
weighted_a_star(2,"input1.txt")

testing weighted A* algorithm for testcase(input1)
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Total Number of States : 28
Total Time : 8
execution time: 0.00125885009765625s



**2nd Testcase:**

In [111]:
weighted_a_star(2,"input2.txt")

testing weighted A* algorithm for testcase(input2)
Path : 9 -> 10 -> 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8
Total Number of States : 51
Total Time : 8
execution time: 0.0020294189453125s



**3rd Testcase:**

In [112]:
weighted_a_star(2,"input3.txt")

testing weighted A* algorithm for testcase(input3)
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11 -> 10
Total Number of States : 48
Total Time : 13
execution time: 0.0013976097106933594s



### Alpha = 3

**1st Testcase:**

In [113]:
weighted_a_star(3,"input1.txt")

testing weighted A* algorithm for testcase(input1)
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Total Number of States : 24
Total Time : 8
execution time: 0.002000570297241211s



**2nd Testcase:**

In [114]:
weighted_a_star(3,"input2.txt")

testing weighted A* algorithm for testcase(input2)
Path : 9 -> 4 -> 12 -> 3 -> 7 -> 5 -> 8 -> 10 -> 8 -> 5 -> 7 -> 5
Total Number of States : 45
Total Time : 13
execution time: 0.0010006427764892578s



**3rd Testcase:**

In [115]:
weighted_a_star(3,"input3.txt")

testing weighted A* algorithm for testcase(input3)
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 10 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11
Total Number of States : 39
Total Time : 14
execution time: 0.001272439956665039s



### Alpha = 4

**1st Testcase:**

In [116]:
weighted_a_star(4,"input1.txt")

testing weighted A* algorithm for testcase(input1)
Path : 1 -> 3 -> 4 -> 5 -> 7 -> 10 -> 11 -> 9 -> 8
Total Number of States : 19
Total Time : 8
execution time: 0.0019991397857666016s



**2nd Testcase:**

In [117]:
weighted_a_star(4,"input2.txt")

testing weighted A* algorithm for testcase(input2)
Path : 9 -> 4 -> 2 -> 10 -> 2 -> 11 -> 3 -> 7 -> 5 -> 8
Total Number of States : 29
Total Time : 9
execution time: 0.0017979145050048828s



**3rd Testcase:**

In [118]:
weighted_a_star(4,"input3.txt")

testing weighted A* algorithm for testcase(input3)
Path : 13 -> 11 -> 10 -> 3 -> 2 -> 6 -> 12 -> 5 -> 10 -> 5 -> 9 -> 4 -> 1 -> 13 -> 11
Total Number of States : 39
Total Time : 14
execution time: 0.0020003318786621094s



## Table Results

### BFS Results :
| --- | Min Spent Time | Total States seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 33 | 0.00134s |
| Test 2 | 8 | 88 | 0.00266s |
| Test 3 | 13 | 844 |  0.10469s |

### IDS Results :
| --- | Min Spent Time | Total States seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 914 | 0.00403s |
| Test 2 | 8 | 3793 | 0.01483s |
| Test 3 | 13 | 68476 | 0.21809s |

### A* Results :
| --- | Min Spent Time | Total States seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 34 | 0.00100s |
| Test 2 | 8 | 53 | 0.00396s |
| Test 3 | 13 | 459 | 0.02738s |

### Weighted A* 2 Results:
| --- | Min Spent Time | Total States Seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 28 | 0.00109s |
| Test 2 | 8 | 51 | 0.00099s |
| Test 3 | 13 | 48 | 0.00200s |

### Weighted A* 3 Results:
| --- | Min Spent Time | Total States seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 24 |  0.00099s |
| Test 2 | 12 | 45 | 0.00218s |
| Test 3 | 14 | 39 | 0.00100s |

### Weighted A* 4 Results:
| --- | Min Spent Time | Total States seen | Avg Exec Time |
| --- | --- | --- | --- |
| Test 1 | 8 | 19 |  0.00101s |
| Test 2 | 9 | 29 | 0.00201s |
| Test 3 | 14 | 39 | 0.00096s |

## Advantage and Disadvantages

### BFS
- Avdantages:
    - If there is a solution for the problem, BFS can find it
    - It will find optimal solution if there are several anwers for the problem
    - BFS will never get trapped in a blind alley, which means unwanted nodes
- Disadvangese:
    - It stores all parents node to complete search,so it gets a lot of memory
    - If a solution is far away then it consumes time
    
    
### IDS
- Advantages:
    - If there is a solution for the problem, IDS can find it
    - It will find optimal solution if there are several anwers for the problem
    - It takes much less memory than BFS
- Disadvantages:
    - If a solution is far away then it consumes time
    
    
### A*
- Advantages:
    - If there is a solution for the problem, A* can find it
    - It will find optimal solution if there are several anwers for the problem
    - It expandes far fewer nodes than BFS and IDS
- Disadvantages:
    - Big tasks are difficult to put in algorithms
    
    
### Weighted  A*
- Advantages:
    - if there is a solution for the problem, weighted A* can find it
    - it takes fewer steps compare to A*
    - execution time is less than A*
- Disadvantages:
    - depending on the Alpha, algorithm is no more optimal